# Aninmal Shelter - Dog Classification

In [41]:
# Imports
import pandas as pd
import os

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

## Data Preparation

In [4]:
labels_path = 'data/labels.csv'
labels=pd.read_csv(labels_path)

In [ ]:
# List of filenames
image_dir = 'data/train/'
filenames = [image_dir + fname + '.jpg' for fname in labels['id']]

# Check if any files do not exist
missing_files = [fname for fname in filenames if not os.path.isfile(fname)]
print(f"Missing files: {len(missing_files)}")

Missing files: 0


In [ ]:
# Set variables
X = filenames
y = labels['breed']

In [52]:
# One Hot Encode
encoder = OneHotEncoder(sparse_output=False)
y_encoded = encoder.fit_transform(labels[['breed']])
encoding_labels = encoder.categories_[0]

In [55]:
# Split data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

# Subset for building
X_train_subset = X_train[:1000]
X_val_subset = X_val[:1000]